# 01 — ASR и идентификация говорящего

Цель ноутбука — показать базовый прототип:
- локального распознавания речи (ASR) с помощью `faster-whisper` (и при желании Vosk);
- извлечения эмбеддингов говорящего с помощью `speechbrain`;
- простого мэппинга говорящего к пользователю и его пресетам.

Допущения:
- работаем оффлайн, без платных API;
- целевая платформа — ноутбук (1660 Ti / Macbook M‑серии) и сервер с/без GPU.

В конце ноутбука должны быть:
- функция для обработки аудиофайла/чанка и возврата `[text, speaker_embedding]`;
- простой пример базы пользователей и сопоставления эмбеддингов с пользователями.


In [ ]:
# Базовые импорты и конфигурация

import os
from pathlib import Path

import numpy as np

# ASR
from faster_whisper import WhisperModel

# Speaker embeddings
from speechbrain.pretrained import EncoderClassifier


DEVICE = "cuda" if os.environ.get("USE_CUDA", "0") == "1" else "cpu"
MODEL_SIZE = "small"  # можно переключать на tiny / base для слабых машин


def load_asr_model(model_size: str = MODEL_SIZE, device: str = DEVICE):
    """Загружает модель faster-whisper для оффлайн ASR."""
    compute_type = "float16" if device == "cuda" else "int8"
    model = WhisperModel(model_size, device=device, compute_type=compute_type)
    return model


def load_speaker_encoder():
    """Загружает энкодер голосовых эмбеддингов (speechbrain)."""
    encoder = EncoderClassifier.from_hparams(
        source="speechbrain/spkrec-ecapa-voxceleb",
        savedir="pretrained_models/spkrec-ecapa-voxceleb",
    )
    return encoder



## Загрузка и предобработка аудио

В этом разделе:
- загружаем аудио-файл локально;
- приводим к моно и частоте дискретизации 16 кГц;
- готовим данные для ASR и энкодера говорящего.


In [ ]:
# Загрузка и предобработка аудио

from typing import Tuple

import librosa
import soundfile as sf


TARGET_SR = 16_000


def load_audio_mono(path: str | Path, target_sr: int = TARGET_SR) -> Tuple[np.ndarray, int]:
    """Загружает аудио в моно, приводит к нужной частоте дискретизации."""
    path = Path(path)
    audio, sr = sf.read(path)

    # Приводим к моно
    if audio.ndim > 1:
        audio = audio.mean(axis=1)

    # Приводим к нужной частоте дискретизации
    if sr != target_sr:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)
        sr = target_sr

    # Нормализация (опционально)
    if np.max(np.abs(audio)) > 0:
        audio = audio / np.max(np.abs(audio))

    return audio.astype("float32"), sr



## Транскрипция и эмбеддинг говорящего

Теперь напишем функции, которые:
- принимают путь к аудио;
- используют одну и ту же волну для ASR и speaker encoder;
- возвращают текст и вектор эмбеддинга.

Для простоты берём весь файл как единый фрагмент (позже можно нарезать на чанки для онлайн-сценария).


In [ ]:
# Транскрипция и эмбеддинг говорящего

import torch


def transcribe_and_embed(
    audio_path: str | Path,
    asr_model: WhisperModel,
    speaker_encoder: EncoderClassifier,
    language: str | None = None,
):
    """Возвращает (полный текст, список сегментов, эмбеддинг говорящего)."""
    audio, sr = load_audio_mono(audio_path, target_sr=TARGET_SR)

    # ASR (faster-whisper поддерживает numpy-массивы)
    segments, info = asr_model.transcribe(
        audio=audio,
        language=language,  # None = автоопределение
    )

    all_segments = list(segments)
    full_text = " ".join(seg.text.strip() for seg in all_segments)

    # Speaker embedding через speechbrain
    waveform = torch.from_numpy(audio).float().unsqueeze(0)  # (1, T)
    with torch.no_grad():
        emb = speaker_encoder.encode_batch(waveform)
    # Обычно размер (1, 1, D) → приводим к (D,)
    emb = emb.squeeze(0).squeeze(0).cpu().numpy()

    return full_text, all_segments, emb



## Простая база пользователей и пресетов

Сделаем очень простую модель базы пользователей:
- у каждого пользователя есть `user_id`, вектор эмбеддинга и словарь пресетов;
- при новом эмбеддинге ищем ближайшего пользователя по косинусному сходству;
- если сходство ниже порога — создаём нового пользователя.

Это заготовка, которую потом можно вынести в отдельный сервис (`user_preset_service`).


In [ ]:
# Простая база пользователей

from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple


@dataclass
class User:
    user_id: str
    embedding: np.ndarray
    presets: Dict[str, str] = field(default_factory=dict)


class UserDB:
    def __init__(self, similarity_threshold: float = 0.7):
        self.users: List[User] = []
        self.similarity_threshold = similarity_threshold

    @staticmethod
    def _cosine_sim(a: np.ndarray, b: np.ndarray) -> float:
        a = a / (np.linalg.norm(a) + 1e-8)
        b = b / (np.linalg.norm(b) + 1e-8)
        return float(np.dot(a, b))

    def add_user(self, embedding: np.ndarray, user_id: Optional[str] = None, presets: Optional[Dict[str, str]] = None) -> User:
        if user_id is None:
            user_id = f"user_{len(self.users) + 1}"
        user = User(user_id=user_id, embedding=embedding, presets=presets or {})
        self.users.append(user)
        return user

    def match_user(self, embedding: np.ndarray) -> Tuple[Optional[User], float]:
        if not self.users:
            return None, 0.0
        best_user = None
        best_sim = -1.0
        for user in self.users:
            sim = self._cosine_sim(embedding, user.embedding)
            if sim > best_sim:
                best_sim = sim
                best_user = user
        if best_sim < self.similarity_threshold:
            return None, best_sim
        return best_user, best_sim



## Пример использования на одном аудиофайле

Ниже — примерный сценарий:
1. Загружаем модели ASR и speaker encoder.
2. Обрабатываем аудиофайл (например, речь Кирилла).
3. Пытаемся найти пользователя в базе; если не нашли — создаём нового с пресетами.

Ты можешь положить свои аудиофайлы в папку `data/audio/` и подставить путь.


In [ ]:
# Пример использования

# Путь к аудио (замени на свой файл, например, data/audio/kirill_1.wav)
audio_path = "data/audio/example_speaker.wav"

# 1. Загружаем модели (может занять время при первом запуске)
asr_model = load_asr_model()
speaker_encoder = load_speaker_encoder()

# 2. Выполняем транскрипцию и получаем эмбеддинг
text, segments, emb = transcribe_and_embed(audio_path, asr_model, speaker_encoder, language=None)
print("Recognized text:\n", text)

# 3. Работаем с базой пользователей
user_db = UserDB(similarity_threshold=0.7)

# Пример заранее добавленного пользователя "Кирилл"
kirill = user_db.add_user(
    embedding=emb,  # в реальности лучше усреднить несколько записей
    user_id="kirill",
    presets={
        "style": "cinematic, high contrast, dark blue palette",
        "character": "male character resembling Kirill",
    },
)

matched_user, sim = user_db.match_user(emb)
print(f"Matched user: {matched_user.user_id if matched_user else None}, similarity={sim:.3f}")

# В дальнейшем пресеты matched_user можно использовать при формировании промпта для генерации изображений.

